This notebook explores the issue of tracing back through data in EPA's systems from facility identifiers to company identifiers. We want to get from specific facilities in the ECHO database to the business entities that operate those facilities such that we can connect other dots and provide more well-rounded information. The initial use case on this is getting from faclities to commercial organizations to political campaign contributions from those organizations to particular campaigns, candidates, members, and committees. While we can go from "facility," which we have in the ECHO data now to contributors in the OpenSecrets data we are exploring, in some cases based on name matching/similarity, this is pretty fraught with potential problems. We need to get at the underlying commercial entities, disambiguate names, incorporate one or more unique identifiers, and come up with much more robust and certain data. From this connection, we will also be able to link out to a lot of other interesting information about particular commercial entities and create more actionable intelligence into our EEW system.

The EPA data service I'm looking at here is the [Facility Registry Service](https://www.epa.gov/frs), which ties in with something called the [Central Data Exchange](https://cdx.epa.gov/About/AboutRegulation). This is where companies regulated through EPA submit and maintain information about their companies and regulated facilities. After exploring some of the functionality through FRS web services, setting up a process that uses downloaded data is likely to work out better for our purposes in the EEW project as we'll want to clear up some disambiguity in the underlying data and connect company entities to other information sources.

The information that we'd want can be downloaded from [this page](https://www.epa.gov/frs/epa-state-combined-csv-download-files), which includes individual state downloads as well as a combined download that is over 1GB in size. The organization information is contained in the "ORGANIZATION_FILE.CSV" for either a specific state or the "NATIONAL" file. This data table links REGISTRATION_ID for a facility, the entities that we are looking at in the EEW scorecard process, to organization/company information. Sometimes these are essentially a one-to-one relationship, but there are many cases (perhaps most) where a given facility is actually a part of some particular company, which may also have subsidiary relationship to some other company. There's also a temporal dynamic to the data and the facility registration process, which gets really messy. I know from previous work I've done in CERCLA and MBTA law enforcement just how challenging it can be to track down exactly what commercial interests have been involved in a given facility through time. I think all of the data we need to get these dots connected is there; it's just going to be a matter of building out a process that makes the best sense of this we can, recording how we get to the linkages we make so that we are transparent in it and can correct any mistakes that come in over time.

The sections in the notebook share some work in progress thinking on how to go about this challenge and where we can obtain data to help. I do reference a data folder that I'm not uploading to GitHub as it contains really large files pulled down from the EPA FRS system.

In [1]:
import requests
import pandas as pd
import numpy as np
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import tabula
from uuid import uuid4

# Data Flow and Provenance Tracing
We need a process that can be run routinely to check for new data, run "dot-connecting" code to link data submitted to the CDX and available through the FRS to other data, and set up derivative services for use in EEW. We need this system to track and record what it's doing through every step so that we can be fully transparent in what's happening in getting from some pretty messy data to usable data. We need this transparency in the process so that we and our users can be confident in the results we are presenting. Any parts of the process that we run introducing potentially undertain information have to be carefully thought through and orchestrated in a way that we can go back and revisit them if we find errors.

For instance, the FRS data on companies has interesting things going on where, through time, a company name may be registered in multiple ways, likely by multiple divisions or people within that company responding to the regulatory requirement. This results in facility names and company names that are slightly or dramatically different but ultimately track back to the same entity. We're also looking here at the DUNS number (Dun & Bradstreet), a global registration process for financial transactions between commercial and non-commercial entities, as a linkable identifier contained in the FRS data. There are cases in the data where DUNS can be used to disambiguate between differently named companies but also cases where multiple divisions within an organization have different DUNS numbers. Anything we do in this kind of space is going to be imperfect, and we need a way of understanding where our imperfections lie and dealing with them based on their potential risk for the end result.

Toward supporting an objective of both a technically robust and scalable solution for data flow with this new data dimension and potential application to the previous work on pulling in the ECHO data, I'm experimenting with the [Pachyderm](https://www.pachyderm.com/) platform. There are lots of data processing pipeline solutions in the open, commercial, and hybrid spaces. I also poked around at [Ploomber](https://github.com/ploomber/ploomber) as a project that is Jupyter Notebook based, which I thought was interesting.

What I've liked about Pachyderm from the start is the way they've baked in versioning and provenance tracking around which everything else is designed. Data are loaded to a Git-based repository and automatically versioned through time. Versions kick off composed pipelines of processing based in Docker containers, scaled on Kubernetes; all pretty standard and well established methods and technologies. Pachyderm is running a [SaaS platform](https://docs.pachyderm.com/latest/hub/hub_getting_started/) that offers free 4-hour workspaces with enough capacity for me to start experimenting with, so I'm getting my head around how this might work in terms of the workflows necessary to fetch FRS data and process them for use.

We're ultimately going to be dealing with a whole bunch of different source data starting points, from the FRS dataset to OpenSecrets data that is part of this use case and other content. Each source has some unique characteristics about how we have to check for updates and retrieve data. Many of them will also have some vagaries and potential issues in terms of break points where something might change in how a source is served up, causing us to need to adjust. We'll want to build in some safeguards and notification services that let us know when something goes wrong along with methods that mitigate for those issues. This needs to come with a thorough understanding of the dependencies between what will be loosely coupled data and processes so that we avoid chain reactions of errors cropping up if some particular process breaks for whatever reason. This is an area where I think implementing some framework like Pachyderm might really help the entire suite of things we are doing in the EEW project.

## Data Source Interfaces
The EPA FRS [REST services](https://www.epa.gov/frs/frs-rest-services) provide some interesting query possibilities, but they are not at all comprehensive into the underlying data. The [query UI](https://www.epa.gov/frs/frs-query) has much more comprehensive information and options, but it's not really machine-actionable. The [download](https://www.epa.gov/frs/epa-state-combined-csv-download-files) is a web page with links to ZIP files, but there's no service interface that tells us when something changes.

As a bit of an aside, this is a real pet peeve of mine with government data services in general. Because of the way these things evolve over time, there are almost no cases of really well built and holistic data distribution architecture that integrates all three elements - API, UI, and bulk download. I've often encouraged an API-first mentality in all things - build every feature in a given system at the API level first, and then build everything else on top of that. This modality helps to force good design throughout an iterative process with built-in integration across what can be a complex landscape of capabilities.

Once we start feeding FRS data into our system, the only real option we'll have is to check, on some sort of routine, the header of the download request for the Last-Modified date against our own log or existing source to determine if we need to trigger a download. In executing this with Pachyderm, this will be a small Dockerized process that essentially references a repo config for a given source, determines if an action is required, and triggers a download to the repo if needed. New data showing up in the repo will then kick off a processing pipeline. I've started composing this and will provide more info and links to Docker containers as I get it working.

In [2]:
frs_national_download_url = "https://ofmext.epa.gov/FLA/www3/state_files/national_combined.zip"
r_head = requests.head(frs_national_download_url)
print(r_head.headers)

{'Date': 'Sun, 12 Sep 2021 16:40:36 GMT', 'Server': 'Oracle-Application-Server-11g', 'Strict-Transport-Security': 'max-age=63072000; includeSubdomains; preload', 'Last-Modified': 'Thu, 02 Sep 2021 15:52:43 GMT', 'ETag': '"5764d739-48ee0c44-5cb052c5b28c0"', 'Accept-Ranges': 'bytes', 'Content-Length': '1223560260', 'Content-Type': 'application/zip', 'Content-Language': 'en', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive'}


# FRS Organization Data
The state and national downloads of FRS data contain a PDF file that documents the contents. This is another pet peeve of mine that points to another principle related to the API-first dynamic described above - everything about your data should be expressed through the API first, and then layer on other representations. If you want to put a PDF file out for people to read about your data, that's fine; just give us something more usable as a foundation.

We will want to have good documentation in our system behind EEW that corrects for this deficiency in areas we can do something about. To help in this, I experimented with several tools and methods for extracting raw content from the 2012-era documentation EPA is including in their FRS download.

Camelot and Tablula-Py both seemed like they might be reasonable options as the documentation is organized in tables. Both of these produce Pandas dataframes for each table. Tables are broken up across pages, which could be concatenated together if we get columns to match up, complicated by the fact that not all tables have a header row with column names. Furthermore, the "Field" property used in the documentation tables doesn't actually line up with the property names in the data tables anyway. Rather, there's a "Source: xxx" tacked onto the end of the descriptions. This is in the form of a source_table.field_name string that tells us essentially how to stitch related tables together.
    
Given the fact that these data are all part of an overall system with multiple distribution points, there's a reasonable chance that descriptions and maybe even other technical details about some of the data are found elsewhere in better formats. The geospatial forms of the data EPA puts out generally come along with FGDC CSDGM metadata in XML format, which should have entity/attribute information we could leverage easier. However, those are functionally completely different datasets with documentation that we can't unequivically say lines up with what we get in the bulk download. We could do some further sleuthing and decide we can use some of it in a different context, but I'm sticking with trying to make sense of the documentation included with the actual data I'm trying to use here.

After experimenting with other methods of extracting PDF text using the textract Python wrapper on more fundamental tools like Tesseract and Poppler, I ended up working with the Tabula-Py package. This creates a still quite messy but at least workable structure that I'll discuss a little bit below.

In [3]:
# Grab the tables from the applicable pages in the FRS documentation file
frs_doc_tables = tabula.read_pdf(
    "data/Facility State File Documentation 11132012_new.pdf",
    pages="15-17",
    stream=True,
    pandas_options={"header": None}
)

Looking at these dataframes, we can see right away that we have some problems. Each dataframe here is a piece of the puzzle describing "3.2.6 Organization File" from the PDF documentation. I set header to none in the pandas options in order to avoid an issue where we get row content as column names when it's the first row of what Tabula reads as a distinct table. We can then get rid of rows with "Field Name" values (or some other filter), and name columns in a way that makes sense.

The only information we really need from these tables is what's in the Description column. The Field Name is also somewhat useful as a plain language title for the field. The Length property isn't really useful as we can derive this from the actual data, and we don't really care anyway. The Type field isn't really useful either, except that we need to use it as a grouping factor. Looking at the extracted output of the dataframes, we see that both Field Name and Description are sometimes broken up across multiple rows based on line breaks and perhaps other formatting in the underlying text that Tabula interprets as distinct rows. We might be able to clean that up with pandas options or java options for the underling library Tabula uses.

However, we can deal with this in another way with some row processing logic in Pandas. Looking at the raw tables, Type is the column that is interpreted here as a value populated for each record and simple enough to only come in once for each logical record. We can treat the first time Type shows up as the first row of a multi-row record, set a unique key value, and then group the values from other rows we want. This gives us a useful hook into the "Source" strings that provide the fields that are actually in our data (e.g., REGISTRY_ID) by setting these out as a distinct member of a grouped list.

In the following codeblock, I run through this whole process, generating out the start to a schema document with keys that should correspond to properties/fields in our data table. I'm somewhat fond of the JSON Schema approach to documenting many different types of tabular data schemas and their properties. It gives us most of the essential documentation elements and has some useful tooling in Python and other languages for data validation and transformation. What I produce here is the stub to this for a properties array in a format that we can easily reference from our data. The source_table attribute here doesn't conform with JSON Schema; it just gives us the hook back to describe actual tables in the source dataset. This same method could be expanded to digest the full PDF document and all table/property descriptions into an overall suite of documentation for the entire schema.

In [4]:
# Fix a couple of issues in the dataframes, pull out usable columns, and concatenate them together
column_names = ["titles","grouper","descriptions"]
usable_dfs = list()
for df in frs_doc_tables[1:]:
    if len(df.columns) > 5:
        df[3] = df.apply(lambda x: x[2] if isinstance(x[2], str) else x[3], axis=1)
        df_usable = df[[1,3,5]]
    else:
        df_usable = df[[1,2,4]]
    df_usable.columns = column_names
    df_usable = df_usable[df_usable.titles != 'Field Name']
    usable_dfs.append(df_usable)
    
df_stitched_tables = pd.concat(usable_dfs)

# Assign a new grouping column as a unique ID and forward fill nulls to give us a grouping factor
df_stitched_tables['uuid'] = df_stitched_tables['grouper'].apply(lambda _: uuid4() if isinstance(_, str) else None)
df_stitched_tables["uuid"] = df_stitched_tables["uuid"].fillna(method="ffill")

# Take care of some issues with empty strings
df_stitched_tables = df_stitched_tables.replace(r'^\s*$', np.nan, regex=True)

# Group on the new UUID and put tuples together from values in title and description
grouped_fields = df_stitched_tables[["uuid","titles","descriptions"]].groupby("uuid")
df_frs_org_documentation = grouped_fields.aggregate(lambda x: tuple(x[x.notnull()]))

# Build the start to schema documentation that we can use as a resource to check field definitions
frs_org_schema = dict()
for index, row in df_frs_org_documentation.iterrows():
    property_name_parts = row.descriptions[-1].split(":")[-1].strip().split(".")
    frs_org_schema[property_name_parts[-1]] = {
        "title": " ".join(row.titles),
        "source_table": property_name_parts[0],
        "description": " ".join(row.descriptions[:-1])
    }
    
frs_org_schema

{'REGISTRY_ID': {'title': 'Facility Registry Identifier',
  'source_table': 'FRS_INTEREST',
  'description': 'The identification number assigned by the EPA Facility Registry System to uniquely identify a facility site.'},
 'STATE_NAME': {'title': 'Mailing Address State Name',
  'source_table': 'FRS_MAILING_ADDRESS',
  'description': 'The name of the state where mail is delivered.'},
 'EIN': {'title': 'Employer Identifier',
  'source_table': 'FRS_ORGANIZATION',
  'description': 'The unique tax identification number issued by the Internal Revenue Service to the employer.'},
 'STATE_CODE': {'title': 'Mailing Address State Code',
  'source_table': 'FRS_MAILING_ADDRESS',
  'description': 'The U.S. Postal Service abbreviation that represents the state or state equivalent for the U.S. and Canada.'},
 'MAILING_ADDRESS': {'title': 'Mailing Address Text',
  'source_table': 'FRS_MAILING_ADDRESS',
  'description': 'The exact address where a mail piece is intended to be delivered, including urban-s

If we take a quick peak at the columns in our giant table of organizations, we can see that we've been able to successfully extract a documentation structure that aligns with column names.

In [5]:
# Grab the first row of the organizations file to get it's columns
df_frs_orgs_peak = pd.read_csv(
    "data/NATIONAL_ORGANIZATION_FILE.CSV",
    nrows=1
)

for col_name in df_frs_orgs_peak.columns:
    print(col_name)
    if col_name in frs_org_schema:
        display(frs_org_schema[col_name])

REGISTRY_ID


{'title': 'Facility Registry Identifier',
 'source_table': 'FRS_INTEREST',
 'description': 'The identification number assigned by the EPA Facility Registry System to uniquely identify a facility site.'}

PGM_SYS_ACRNM


{'title': 'Information System Abbreviated Name',
 'source_table': 'FRS_AFFILIATION',
 'description': 'The abbreviated name that represents the name of an information management system for an environmental program. Program System Definitions can be viewed at http://www.epa.gov/enviro/html/fii/prog_sys.html. Source:'}

PGM_SYS_ID


...

{'title': 'Mailing Address State Name',
 'source_table': 'FRS_MAILING_ADDRESS',
 'description': 'The name of the state where mail is delivered.'}

POSTAL_CODE


{'title': 'Mailing Address Postal Code',
 'source_table': 'FRS_MAILING_ADDRESS',
 'description': 'The combination of the five-digit Zone Improvement Plan (ZIP) code and the four-digit extension code (if available) that represents the geographic segment that is a subunit of the ZIP Code, assigned by the U.S. Postal Service to a geographic location to facilitate mail delivery; or the postal zone specific to the country, other than the U.S., where the mail is delivered.'}

COUNTRY_NAME


{'title': 'Mailing Address Country Name',
 'source_table': 'FRS_MAILING_ADDRESS',
 'description': 'The name of the country where the addressee is located. Default: United States'}

# Examining Organization (Company) Identifiers
The FRS data provides essentially three different identifiers that give us a chance to hook these records to something larger from which we can link effectively across disparate datasets, like the information in the campaign finance records from OpenSecrets that is an initial motivating use case.
* DUNS_NUMBER from the [Dun and Bradstreet](https://www.dnb.com/) registry
* EIN - Employer Identifier Number associated with payrolls and taxes
* STATE_BUSINESS_ID that may be usable with a state by state analysis of other sources

These identifiers are not universally populated in the data, and we have no real way of evaluating quality of the information just by looking at what's provided in the FRS other than basic consistency checks, which may not mean much. We also have organization names, some of which might be fairly straightforward but come with ambiguity problems, and other clues like addresses that might be combined with other information to help pin things down.

Each of these identifiers comes with its own challenges. There are a lot of incentives to keeping information about commercial companies, particularly privately held firms, out of the public domain or at least obfuscated. Companies like D&B have a whole business model built around building up a proprietary database of commercial entity information. The reason their DUNS numbers are in places like the EPA database is that they provide a hook into that proprietary system that other organizations buy into and use for various purposes. For instance, if you apply for an NSF grant you almost always have to have a DUNS number associated with your university or lab or even government organization that is then used to do things like check on financial standing, audit status, risk analysis, and other details that get tied back into the DUNS system.

EIN numbers can be accessible for companies registered with the SEC, but they are much more difficult to find for private companies. US State Secretaries of State offices are generally the agency regulating businesses, and we might be able to hook on the state business IDs included in the FRS data. However, each one of those systems is very different with widely varying interfaces and fragmented availability of data.

I had some hope of using Wikidata as a neutral and publicly accessible arbiter. Information on elected members of Congress is pretty complete in Wikidata through time and includes FEC identifiers that we find in the OpenSecrets campaign finance data. I run through this below, but you'll see some less than satisfactory results. We might or might not be able to start making our own contributions to Wikidata as a place to house linked data.

Philosophically, I like the elegance of using Wikidata and data models already established to house parts of our overall data/information model as a resource we tap to drive different kinds of data integration pipelines. It would put more information into the public domain that many other groups can also take advantage of, and we might find kindred spirits there to all jump in on contributions together. However, the WikiMedia Foundation's own operating principles can serve to thwart this approach in that the community rigorously monitors itself for cases of copyright infringement or anything like it. New records for entities and claims on entities should be referenced to a source that can be verified as already in the public domain, with Wikidata just a place where all that information is coming together. Running wholesale processes to populate Wikidata with information of questionable provenance can be met with getting booted off the system, even though it happens pretty frequently.

In [6]:
%%time
# Grab the whole dataset on REGISTRY_ID and just the columns to examine DUNS_NUMBER
df_frs_orgs = pd.read_csv(
    "data/NATIONAL_ORGANIZATION_FILE.CSV", 
    index_col=["REGISTRY_ID"],
    usecols=["REGISTRY_ID","ORG_NAME","ORG_TYPE","DUNS_NUMBER","EIN"],
    low_memory=False
)

CPU times: user 8.92 s, sys: 2.28 s, total: 11.2 s
Wall time: 15.3 s


## DUNS Numbers
I started here with a check to see if we can find any of our companies with DUNS numbers using the [DUNS identifier property](https://www.wikidata.org/wiki/Property:P2771) in Wikidata. I first pull out a dataframe with just those organization records populated with DUNS_NUMBER and then consult Wikidata.

Just looking at the head here from the FRS data, we can see that we have interesting issues like a very slight variation in one company name and two different DUNS numbers for what is quite likely the same entity. These might have come into the EPA CDX registry at different times. They both point to the same REGISTRY_ID value. We may or may not be able to completely decipher what's going on with what is a very large database.

In [7]:
df_companies_with_duns = df_frs_orgs[df_frs_orgs.DUNS_NUMBER.notnull()]
print(len(df_companies_with_duns.DUNS_NUMBER.unique()))
display(df_companies_with_duns.head())

21130


,ORG_NAME,ORG_TYPE,DUNS_NUMBER,EIN
REGISTRY_ID,,,,
110000491735,MARATHON PETROLEUM CORP,PRIVATE,968393996,NaN
110000491735,PETRO STAR INC.,PRIVATE,131463705,NaN
110000491744,CHEVRON USA INC,PRIVATE,009140559,NaN
110000491762,"UNIVAR SOLUTIONS USA, INC",UNKNOWN,181523838,NaN
110000491762,UNIVAR SOLUTIONS USA INC.,UNKNOWN,102971785,NaN


Wikidata can be queried through its SPARQL end point, which is decipherable through its [query builder](https://query.wikidata.org/). SPARQL is a very low level interface that is accessing very granulary data in triples. Complex SQPARQL queries can be crafted to go after data in interesting ways, but it's not a tech that I own very well at all. I mostly have found it easier to go after a batch of records, pull those into memory or some other data structure, and then work with that from there.

At the end of the day, Wikidata is a really simple model that can contain very rich information when well-populated. For what we're interested in here, it's basically just entities and claims/statements. Entities have nothing more than an identifier (persistent and resolvable), name, description, and aliases in different languages. Claims (or statements) are linked to entities, some of which are in a special category of identifiers as in the case of our DUNS numbers (and [EIN](https://www.wikidata.org/wiki/Property:P1297), which we'll get to later). All claims are based on a registered property that is also in Wikidata with a unique, persistent, and resolvable identifier. Properties are the chief governance mechanism in Wikidata, with a review and community acceptance process based on a meritocratic system of proven contributions. Property values can point to other Wikidata entities or to some other type of value that is controlled by the characteristics and rules set for the property.

In the codeblock below, I lay out a function for building a query for Wikidata entities that have a particular property to return the Wikidata ID, entity name, and value of the requested.

In [8]:
# Need to work out what I can set up as a more referenceable source for apps like this
user_agent = "CompanyNameDisambiguator/0.1 (https://github.com/skybristol; skybristol@github) pydisambig/0.1"

# Property mapping for convenience and reference
property_mapping = {
    "DUNS_NUMBER": "https://www.wikidata.org/wiki/Property:P2771",
    "EIN": "https://www.wikidata.org/wiki/Property:P1297"
}

def get_wd_by_property(
    property_name, 
    endpoint_url="https://query.wikidata.org/sparql", 
    user_agent=user_agent,
    return_format="dataframe"
):
    if property_name not in property_mapping:
        return

    query = """SELECT
      ?item ?itemLabel
      ?prop
    WHERE 
    {
      ?item wdt:%s ?prop
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    """ % property_mapping[property_name].split(":")[-1]
    
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    wikidata_results = sparql.query().convert()
    
    if return_format == "raw":
        return wikidata_results

    dataset = [
        {
            "wd_url": i["item"]["value"],
            "name": i["itemLabel"]["value"],
            property_name: i["prop"]["value"]
        } for i in wikidata_results["results"]["bindings"]
    ]
    
    if return_format == "dataframe":
        return pd.DataFrame(dataset)

    return dataset



In [9]:
df_duns_in_wikidata = get_wd_by_property("DUNS_NUMBER", return_format="dataframe")

At this point, we have 171 entities that probably are all companies of one kind or another with DUNS number values as identifiers in Wikidata. That's not very many to begin with. When we look at our huge set of FRS data with over 20K references to unique DUNS numbers, we only get 4 matches on pretty well-known companies. Bummer!

In [10]:
display(df_duns_in_wikidata)
display(df_duns_in_wikidata[df_duns_in_wikidata.DUNS_NUMBER.isin(df_companies_with_duns.DUNS_NUMBER.unique())])

,wd_url,name,DUNS_NUMBER
0,http://www.wikidata.org/entity/Q72314763,The Textile Workshop,218140170
1,http://www.wikidata.org/entity/Q107560167,ARCap REIT Inc.,13962951
2,http://www.wikidata.org/entity/Q97119297,Memo Bank,269529865
3,http://www.wikidata.org/entity/Q97824053,Biosciences Research Support Foundation,851737506
4,http://www.wikidata.org/entity/Q98446892,DuckDuckGo Inc.,012746047
...,...,...,...
166,http://www.wikidata.org/entity/Q4040469,Headhunter,683397278
167,http://www.wikidata.org/entity/Q22671360,Alcatel-Lucent Enterprise,491320359
168,http://www.wikidata.org/entity/Q44124288,"Best Fishing In Alaska, LLC",080967295
169,http://www.wikidata.org/entity/Q50359906,CAD Studio (Arkance Systems),495048469


,wd_url,name,DUNS_NUMBER
14,http://www.wikidata.org/entity/Q6792378,Maui Land & Pineapple Company,009132184
73,http://www.wikidata.org/entity/Q329953,Northrop Grumman,967356127
161,http://www.wikidata.org/entity/Q7240,Lockheed Martin,834951691
163,http://www.wikidata.org/entity/Q206921,Pfizer,001326495


## Employer Identification Numbers
We can now run the same process to look for EIN, which yields only slightly better results. We first have to split EIN values from FRS data on their first two characters to line up with the format used in Wikidata. Even though we have many more Wikidata entities with EIN identifiers, we still only end up with a handful that we can directly match to organization records in our EPA FRS data. We'll need to go after a different source and do some more sleuthing to make sure we haven't done anything weird in processing.

In [11]:
def ein_to_string(ein):
    if str(ein) == "0":
        return
    return "-".join([str(ein)[:2],str(ein)[2:]])

df_frs_orgs['EIN_STR'] = df_frs_orgs["EIN"].apply(ein_to_string)

df_companies_with_ein = df_frs_orgs[(df_frs_orgs.EIN_STR != "na-n") & (df_frs_orgs.EIN_STR.notnull())]
len(df_companies_with_ein.EIN_STR.unique())

25582

In [12]:
df_ein_in_wikidata = get_wd_by_property("EIN", return_format="dataframe")

In [13]:
display(df_ein_in_wikidata)
display(df_ein_in_wikidata[df_ein_in_wikidata.EIN.isin(df_companies_with_ein.EIN_STR.unique())])

,wd_url,name,EIN
0,http://www.wikidata.org/entity/Q43449,Creative Commons,04-3585301
1,http://www.wikidata.org/entity/Q35476,AT&T,20-8686806
2,http://www.wikidata.org/entity/Q49135,Isabella Stewart Gardner Museum,04-2104334
3,http://www.wikidata.org/entity/Q114071,Perkins School for the Blind,04-2103616
4,http://www.wikidata.org/entity/Q128316,Honolulu Museum of Art,99-0079713
...,...,...,...
11760,http://www.wikidata.org/entity/Q54559891,Habitat for Humanity of Pinellas County,59-2509116
11761,http://www.wikidata.org/entity/Q54624094,Goodwill Industries of South Florida,59-0866126
11762,http://www.wikidata.org/entity/Q54628764,Long Island Alzheimer's Foundation,11-2926958
11763,http://www.wikidata.org/entity/Q54629520,Greensboro Jewish Federation,23-7107693


,wd_url,name,EIN
353,http://www.wikidata.org/entity/Q7763221,The Seeing Eye,22-1539721
406,http://www.wikidata.org/entity/Q312,Apple Inc.,94-2404110
651,http://www.wikidata.org/entity/Q54557427,Boy Scouts of America National Council,22-1576300
861,http://www.wikidata.org/entity/Q54834384,Griffin Pond Animal Shelter,24-0831491
1407,http://www.wikidata.org/entity/Q66,Boeing,91-0425694
1408,http://www.wikidata.org/entity/Q37156,IBM,13-0871985
2094,http://www.wikidata.org/entity/Q54559868,Habitat for Humanity of San Antonio,74-1897502
2100,http://www.wikidata.org/entity/Q54559861,Dallas Area Habitat for Humanity,75-2097161
2799,http://www.wikidata.org/entity/Q104610148,Baptist Standard,75-0800610
3002,http://www.wikidata.org/entity/Q5357475,Electric Reliability Council of Texas,74-2587416
